# Mathematical Engineering - Financial Engineering, FY 2024-2025

<hr>

## Risk Management - Exercise 1: Hedging a Swaption Portfolio

In [1]:
# Importing relevant libraries. The bootstrap_py package has been built in the previous assignment.
# The ex1_utilities file contains some useful functions.

from bootstrap_py import Bootstrap
from utilities.ex1_utilities import (
    business_date_offset,
    swaption_price_calculator,
    date_series,
    irs_proxy_duration,
    swap_par_rate,
    swap_mtm,
    SwapType,
    shift_curve
)
import pandas as pd
import numpy as np

First, we run the bootstrap on the Market Data, using the `Bootstrap.fit()` function

In [2]:
b = Bootstrap.from_xls("MktData_CurveBootstrap.xls")

dates, discounts = b.fit()

Our portfolio is made up of one $10y - 5y$ swaption and one $10y$ interest rate swap *IRS*. In the following we define some important characteristics

In [3]:
# Parameters
swaption_maturity_y = 10  # In years
swaption_maturity_m = 1  # In months
swaption_tenor_y = 5  # In years
swaption_fixed_leg_freq = 1  # Times a year the fixed leg pays
swaption_type = SwapType.RECEIVER
swaption_notional = 700_000_000
sigma_black = 0.7955  # Black swaption volatility

irs_maturity = 10  # In years
irs_fixed_leg_freq = 1
irs_notional = 600_000_000

# In the following, it's convenient to format the discount factors
# into a pandas Series, with the dates as index.
discount_factors = pd.Series(discounts, index=dates)
today = discount_factors.index[0] # 2023-02-02

### 1. Mark to Market the portfolio, mid-rate curve

In this section we price our instruments, according to the mid-rate curve. First, we compute the forward swap rate, using the function defined in `ex1_utilities.py` called `swap_par_rate()`. This method can compute the forward and the spot swap rate; for a more detailed explanation, please refer to the documentation.

Notice that dates are always checked to be business days, using the provided method `business_date_offset()`.

In [4]:
# Q1: Portfolio MtM
# Computing the forward swap rate

swaption_expiry = business_date_offset(
    today, year_offset=swaption_maturity_y, month_offset=swaption_maturity_m
)

underlying_expiry = business_date_offset(
    today,
    year_offset=swaption_maturity_y + swaption_tenor_y,
    month_offset=swaption_maturity_m,
)

swaption_underlying_fixed_leg_schedule = date_series(
    swaption_expiry, underlying_expiry, swaption_fixed_leg_freq
)

fwd_swap_rate = swap_par_rate(
    swaption_underlying_fixed_leg_schedule[1:],
    discount_factors,
    swaption_underlying_fixed_leg_schedule[0],
)
print(f"Forward swap rate: {fwd_swap_rate:.5%}")

Forward swap rate: 2.97979%


To price the swaption we use `swaption_price_calculator()`, implemented in the utils fils. This method compute also the swaption delta, this information will be useful in the following.

In [5]:
# Q1: Portfolio MtM
# Pricing ATM swaption

strike = fwd_swap_rate  # Since ATM

swaption_price, swaption_delta = swaption_price_calculator(
    fwd_swap_rate,
    strike,
    today,
    swaption_expiry,
    underlying_expiry,
    sigma_black,
    swaption_fixed_leg_freq,
    discount_factors,
    swaption_type,
    compute_delta=True,
)

print(f"Swaption price (single): €{swaption_price:.5f}")
print(f'Swaption price (notional): €{swaption_price*swaption_notional:.2f}')
print(f'Swaption delta: {swaption_delta:.6f}')

Swaption price (single): €0.08152
Swaption price (notional): €57062717.42
Swaption delta: -0.356064


The total value of our portfolio is the swaption plus the IRS. However, the IRS is issued at par, so its Net Present Value is zero. We verify this statement computing the `irs_mtm` through the `swap_mtm()` method.

In [6]:
# Q1: Portfolio MtM
# Calculating portfolio market value

irs_expiry = business_date_offset(today, year_offset=irs_maturity)

# IRS fixed payment dates
irs_fixed_leg_payment_dates = date_series(today, irs_expiry, irs_fixed_leg_freq)[1:]

irs_rate_10y = swap_par_rate(irs_fixed_leg_payment_dates, discount_factors)

# The swap MtM is zero since entered today, so that portfolio MtM equals the swaption value
irs_mtm = swap_mtm(irs_rate_10y,irs_fixed_leg_payment_dates,discount_factors,SwapType.RECEIVER)

print(f"Swap Rate 10y: {irs_rate_10y:.5%}")
print(f"IRS MtM: €{irs_mtm:,.5f}")

ptf_mtm = swaption_notional * swaption_price + irs_notional * irs_mtm
print(f"Portfolio MtM: €{ptf_mtm:,.2f}")

Swap Rate 10y: 2.85050%
IRS MtM: €-0.00000
Portfolio MtM: €57,062,717.42


### 2. Portfolio DV01 Parallel Shift

We evaluate the portfolio sensitivity to a $1bp$ parallel shift of the interest rate curve. To do so, we must first bump the curve up, using the `shift_curve()` function.

In [7]:
# Q2: Portfolio DV01-parallel
# Bootstrap after a shock on market rates

b_shocked = shift_curve(b, bp=1e-4)
_, discounts_shocked = b_shocked.fit()
discount_factors_up = pd.Series(discounts_shocked, index=dates)

Now, we re-evaluate our positions: first the forward swap rate, then the swaption and finally the IRS. The latter won't be zero anymore, because we've shifted the curve.

In [8]:
# Q2: Portfolio DV01-parallel
# Re-evaluate swaption price

fwd_swap_rate_up = swap_par_rate(
    swaption_underlying_fixed_leg_schedule[1:],
    discount_factors_up,
    swaption_underlying_fixed_leg_schedule[0],
)

# Swaption shocked price
swaption_price_up, delta_up = swaption_price_calculator(
    fwd_swap_rate_up, # ERROR FWD_SWAP_RATE
    strike,
    today,
    swaption_expiry,
    underlying_expiry,
    sigma_black,
    swaption_fixed_leg_freq,
    discount_factors_up,
    swaption_type,
    compute_delta=True
)


print(f"Price Swaption (single) Up: {swaption_price_up:.5f}")
print(f"Price Swaption (notional) Up: {swaption_price_up*swaption_notional:.2f}")

Price Swaption (single) Up: 0.08138
Price Swaption (notional) Up: 56965451.95


In [9]:
# Q2: Portfolio DV01-parallel
# Re-evaluate IRS MtM

# Swap shocked MtM ~ no longer zero
irs_mtm_up = swap_mtm(
    irs_rate_10y,
    irs_fixed_leg_payment_dates,
    discount_factors_up
)

print(f"Price IRS (single) Up: €{irs_mtm_up:,.5f}")
print(f"Price IRS (notional) Up: €{irs_mtm_up*irs_notional:,.2f}")

Price IRS (single) Up: €0.00086
Price IRS (notional) Up: €514,969.19


To calculate the DV01 we just need to take the difference between the two portfolio values. We also compute the DV01 of each instrument, as will later on be convenient in the hedging process.

In [10]:
# Q2: Portfolio DV01-parallel
# Compute DV01s

# Portfolio shocked MtM
ptf_mtm_up = swaption_notional * swaption_price_up + irs_notional * irs_mtm_up

# DV01
ptf_numeric_dv01 = ptf_mtm_up - ptf_mtm

DV01_swaption = swaption_price_up - swaption_price
DV01_irs = irs_mtm_up - irs_mtm
DV01_portfolio = swaption_notional*DV01_swaption + irs_notional*DV01_irs
print(f"Swaption DV01: {swaption_notional*DV01_swaption:,.2f}")
print(f"IRS DV01: {irs_notional*DV01_irs:,.2f}")
print(f"Portfolio DV01: {DV01_portfolio:,.2f}")

Swaption DV01: -97,265.47
IRS DV01: 514,969.19
Portfolio DV01: 417,703.71


As expected, the DV01 of a receiver swaption is negative, because an increase in the rates will lead to higher floating leg payments for the holder. On the other hand, the DV01 of a payer IRS is positive, because the floating leg payments, which the holder gets, will increase. The total result on the portfolio is a combination of the two effects, leading to a positive DV01 as IRS play a dominant role in the portfolio sensitivity.

### 3. DV01 Approximation

We also estimated the DV01 of the portfolio using an analytical approximation, which considers the swaption $\Delta$ and the IRS Duration $MacD$. The $\Delta$ has been computed some cells above, while the duration is calculated using the `irs_proxy_duration()` function.

In [11]:
# Q3: Analytical portfolio DV01
# DV01 approximation

irs_duration = irs_proxy_duration(
    today, irs_rate_10y, irs_fixed_leg_payment_dates, discount_factors
)

ptf_proxy_dv01 = (
    swaption_notional * swaption_delta + irs_notional * irs_duration
) * 1e-4

print(f"Swaption DV01: {swaption_delta*swaption_notional/10000:,.2f}")
print(f"IRS DV01: {irs_duration*irs_notional/10000:,.2f}")
print(f"Portfolio proxy DV01: €{ptf_proxy_dv01:,.2f}")

Swaption DV01: -24,924.50
IRS DV01: 530,630.66
Portfolio proxy DV01: €505,706.16


The errors of this estimate are quite large in absolute terms. However, we should consider them relatively to the scale of our portfolio, with a total notional of $1.3$ billions. Also, recall that the duration is actually an approximation of the $DV01z$, which for a vanilla IRS is in turn an approximation of the actual DV01.

### 4. Delta-Hegde with 10y IRS

We now proceed to make our porfolio $\Delta$-neutral, using a $10y$ IRS. The total $DV01$ of the portfolio is the sum of the swaption and the IRS ones, weighted for their respective notionals. If we add $x$ units of IRS to the portfolio, the new DV01 will be

$$
    N_{swap}DV01_{swap} + N_{irs}DV01_{irs} + x DV01_{irs} = 0
$$

In [12]:
# Q4: Delta hedging of the swaption changing the IRS notional
# Exact approach

min_lot = 1_000_000

x = -(DV01_swaption*swaption_notional+DV01_irs*irs_notional)/DV01_irs
# 10y swaps are sold in mininum lots of 1M
x = np.round(x/min_lot)*min_lot

delta_hedge_swap_notional = irs_notional + x

delta_hedge_dv01 = (
    swaption_notional * swaption_price_up + delta_hedge_swap_notional * irs_mtm_up
) - ptf_mtm

print(
    f"With €{delta_hedge_swap_notional:,.0f} swap notional the DV01 is €{delta_hedge_dv01:,.0f}."
)

With €113,000,000 swap notional the DV01 is €-280.


The DV01 is not exactly zero, because we had to truncate the exact numbers of contracts, because the are sold in mininum lots of $1 Mln$ each. Notice the total notional of payer $IRS$ has decreased from $600 Mln$ to $113 Mln$, so we are effectively selling payer IRS. This is coherent with our previous observation, the portfolio DV01 is positive, so to rebalance it we need to enter into a *DV01-negative* position, like shorting a payer IRS.

In [13]:
# Q4: Delta hedging of the swaption changing the IRS notional
# Approximate approach

# Using the approximated DV01
# delta_hedge_swap_notional_proxy = -(swaption_delta*swaption_notional+irs_duration*irs_notional)/irs_duration
x = -(swaption_delta*swaption_notional+irs_duration*irs_notional)/irs_duration
x = np.round(x/min_lot)*min_lot

delta_hedge_swap_notional_proxy = irs_notional + x

delta_hedge_dv01_approx = (
    swaption_notional * swaption_price_up + delta_hedge_swap_notional_proxy * irs_mtm_up
) - ptf_mtm

print(
    f"With €{delta_hedge_swap_notional_proxy:,.0f} swap notional the DV01 is €{delta_hedge_dv01_approx:,.0f}."
)

With €28,000,000 swap notional the DV01 is €-73,234.


### 5. Coarse-Grained Bucket DV01

We evaluate the portfolio coarse-grained buckets DV01 for 10 and 15 years. We need to bumb again the interest rate curve, introducing some weights to select the relevant part of the curve. Later on, we re-price the swaption and the IRS, computing the difference

In [14]:
b_bucket_10 = shift_curve(b, bp=1e-4, swaps=False)

b_bucket_10.swaps.loc[:pd.Timestamp('2033-02-02')] = b.swaps[:pd.Timestamp('2033-02-02')] + 0.0001
weights_10y = [(3-(1/5)*(anno+10))*0.0001 for anno in range(6)]

b_bucket_10.swaps[pd.Timestamp('2033-02-02'):pd.Timestamp('2038-02-02')] = (b.swaps[pd.Timestamp('2033-02-02'):pd.Timestamp('2038-02-02')].T + [weights_10y[i] for i in [0,1,2,5]]).T

_, discounts_bucket_10 = b_bucket_10.fit()
discount_factors_buck_10 = pd.Series(discounts_bucket_10, index=dates)

In [15]:
# Q5: Coarse-Grained Bucket DV01
# Re-evaluate swaption price

fwd_swap_buck_10 = swap_par_rate(
    swaption_underlying_fixed_leg_schedule[1:],
    discount_factors_buck_10,
    swaption_underlying_fixed_leg_schedule[0],
)

swaption_price_buck_10, delta_up = swaption_price_calculator(
    fwd_swap_buck_10, # ERROR FWD_SWAP_RATE
    strike,
    today,
    swaption_expiry,
    underlying_expiry,
    sigma_black,
    swaption_fixed_leg_freq,
    discount_factors_buck_10,
    swaption_type,
    compute_delta=True
)

# Swap shocked MtM
irs_mtm_buck_10 = swap_mtm(irs_rate_10y, irs_fixed_leg_payment_dates, discount_factors_buck_10)

# Portfolio shocked MtM
ptf_mtm_buck_10 = swaption_notional * swaption_price_buck_10 + irs_notional * irs_mtm_buck_10

# DV01
ptf_numeric_buck_10 = ptf_mtm_buck_10 - ptf_mtm

print(f"Portfolio BUCK_10: €{ptf_numeric_buck_10:,.2f}")

Portfolio BUCK_10: €559,114.34


We repeate the same procedure, but the 15 year maturity

In [16]:
b_bucket_15 = shift_curve(b, bp=1e-4, depos=False, futures=False, swaps=False)

weights_15y = [(-2+(1/5)*(anno+10))*0.0001 for anno in range(6)]
b_bucket_15.swaps.values[8:11,0] = b.swaps.values[8:11,0] + weights_15y[:3]
b_bucket_15.swaps.values[8:11,1] = b.swaps.values[8:11,1] + weights_15y[0:3]

b_bucket_15.swaps.values[11:] = b.swaps.values[11:] + 0.0001


_, discounts_bucket_15 = b_bucket_15.fit()
discount_factors_buck_15= pd.Series(discounts_bucket_15, index=dates)

In [17]:
fwd_swap_buck_15 = swap_par_rate(
    swaption_underlying_fixed_leg_schedule[1:],
    discount_factors_buck_15,
    swaption_underlying_fixed_leg_schedule[0],
)

swaption_price_buck_15, delta_up = swaption_price_calculator(
    fwd_swap_buck_15, # ERROR FWD_SWAP_RATE
    strike,
    today,
    swaption_expiry,
    underlying_expiry,
    sigma_black,
    swaption_fixed_leg_freq,
    discount_factors_buck_15,
    swaption_type,
    compute_delta=True
)

# Swap shocked MtM
irs_mtm_buck_15 = swap_mtm(irs_rate_10y, irs_fixed_leg_payment_dates, discount_factors_buck_15)

# Portfolio shocked MtM
ptf_mtm_buck_15 = swaption_notional * swaption_price_buck_15 + irs_notional * irs_mtm_buck_15

# DV01
ptf_numeric_buck_15 = ptf_mtm_buck_15 - ptf_mtm

print(f"Portfolio BUCK_15: €{ptf_numeric_buck_15:,.2f}")

Portfolio BUCK_15: €-140,977.46


The two bucket-DV01 have opposite signs, this reflects the fact that the two instruments which make up our portfolio, the swaption and the IRS, have different sensitivities to the interest rate curve. The IRS has a positive DV01, while the swaption has a negative one. The total DV01 is the sum of the two, so the sign of the bucket-DV01 depends on the relative weight of the two instruments in the portfolio. The sum of the two bucket-DV01 is the total DV01 of the portfolio, which we have already computed.

In [18]:
print(f"Sum of the two buckets: {ptf_numeric_buck_10+ptf_numeric_buck_15}")

Sum of the two buckets: 418136.87547539175


### 6. Delta-Hedge with 10y and 15y IRS

Instead of hedging the portfolio with one just IRS, like we did in point 4, we use two different instruments. The most convenient choice is to pick a $10y$ and a $15y$ IRS, because of the reason mentioned above: in fact, we have proven that our portfolio behaves differently in the two buckets, so it's reasonable to try to hedge the two different risk separately.

With two instruments, we have the freedom to select the relative weights. We take the weight of the $10y$ IRS to be exactly opposite to the $10y$ IRS notional of our portfolio, so that our net exporsure on that instrument is null.

The weight of the $15y$ IRS is then determined imposing that the total DV01 is zero

In [19]:
# Q6: Delta hedging w/ two IRS
# IRS notionals

notional_10y = irs_notional

irs_15y_dates = date_series(today, business_date_offset(today, year_offset=15), irs_fixed_leg_freq)[1:]
irs_rate_15y = swap_par_rate(irs_15y_dates, discount_factors)

irs_15y_mtm = swap_mtm(irs_rate_15y, irs_15y_dates, discount_factors_up)

DV01_15y = irs_15y_mtm - 0

notional_15y = -DV01_swaption*swaption_notional/DV01_15y

# min_lots is 1M
notional_10y = np.round(notional_10y/min_lot)*min_lot
notional_15y = np.round(notional_15y/min_lot)*min_lot

print(f"10y IRS notional: -€{notional_10y:,.0f}")
print(f"15y IRS notional: €{notional_15y:,.0f}")

10y IRS notional: -€600,000,000
15y IRS notional: €81,000,000


Now we re-evaluate the coarsed-grained buckets for $10y$ and $15y$. Since the two positions on the swap cancel each other, we don't consider them anymore

In [20]:
# Q6: Delta hedging w/ two IRS
# Bucket DV01 10y

# Swap shocked MtM
irs15y_mtm_buck_10 = swap_mtm(irs_rate_15y, irs_15y_dates, discount_factors_buck_10)

# Portfolio shocked MtM
ptf_mtm_buck_10 = swaption_notional * swaption_price_buck_10 + notional_15y * irs15y_mtm_buck_10

# DV01
ptf_numeric_buck_10 = ptf_mtm_buck_10 - ptf_mtm

print(f"Portfolio BUCK_10: €{ptf_numeric_buck_10:,.2f}")

Portfolio BUCK_10: €44,145.15


In [21]:
# Q6: Delta hedging w/ two IRS
# Bucket DV01 15y

# Swap shocked MtM
irs15y_mtm_buck_15 = swap_mtm(irs_rate_15y, irs_15y_dates, discount_factors_buck_15)

# Portfolio shocked MtM
ptf_mtm_buck_15 = swaption_notional * swaption_price_buck_15 + notional_15y * irs15y_mtm_buck_15

# DV01
ptf_numeric_buck_15 = ptf_mtm_buck_15 - ptf_mtm

print(f"Portfolio BUCK_15: €{ptf_numeric_buck_15:,.2f}")

Portfolio BUCK_15: €-43,457.90


Again, the sum of the two bucket-DV01 is the total DV01 of the portfolio, which, since the portfolio is perfeclty hedge, is pratically zero.

In [22]:
print(f"DV01 Portfolio: {ptf_numeric_buck_10+ptf_numeric_buck_15:,.2f}")

DV01 Portfolio: 687.26


### 7. Curve steepening scenario

We consider a market scenario in which the $10y$ rates decreases by $1bp$, while the $15y$ rates increases by $1bp$. We carry out the Profit&Loss (PnL) computation for the hedged portfolio described before

In [26]:
# Q7: Curve Steepener
# Bootstrap after a shock on market rates

bp = 1e-4
b_steep = Bootstrap.from_xls("MktData_CurveBootstrap.xls")
b_steep.swaps.loc[pd.Timestamp('02-Feb-2033')] -= bp
b_steep.swaps.loc[pd.Timestamp('02-Feb-2038')] += bp

_, discounts_steep = b_steep.fit()
discounts_steep = pd.Series(discounts_steep, index=dates)

In [27]:
# Q7: Curve Steepening
# Re-evaluate swaption price and IRS mtm


# First portfolio: 10y IRS

ptf_0 = swaption_notional * swaption_price + 0     # pre shock

fwd_swap_rate_1 = swap_par_rate(
    swaption_underlying_fixed_leg_schedule[1:],
    discounts_steep,
    swaption_underlying_fixed_leg_schedule[0],
)
swaption_price_1, delta_1 = swaption_price_calculator(
    fwd_swap_rate_1,
    strike,
    today,
    swaption_expiry,
    underlying_expiry,
    sigma_black,
    swaption_fixed_leg_freq,
    discounts_steep,
    swaption_type,
    compute_delta=True
)

irs_mtm_1 = swap_mtm(irs_rate_10y, irs_fixed_leg_payment_dates, discounts_steep)

print(f"Swaption: {swaption_notional * swaption_price_1}")
print(f"IRS: {irs_notional * irs_mtm_1}")

ptf_1 = swaption_notional * swaption_price_1 + irs_notional * irs_mtm_1

print(f'P&L: {ptf_1-ptf_0:,.2f}')

Swaption: 56883339.5877602
IRS: -515282.93425233685
P&L: -694,660.77


In [28]:
# Q7: Curve Steepening
# Re-evaluate swaption price and IRS mtm


# Second portfolio: 10y IRS + 15y IRS
ptf_0 = swaption_notional * swaption_price + 0

fwd_swap_rate_2 = swap_par_rate(
    swaption_underlying_fixed_leg_schedule[1:],
    discounts_steep,
    swaption_underlying_fixed_leg_schedule[0],
)
swaption_price_2, delta_2 = swaption_price_calculator(
    fwd_swap_rate_2,
    strike,
    today,
    swaption_expiry,
    underlying_expiry,
    sigma_black,
    swaption_fixed_leg_freq,
    discounts_steep,
    swaption_type,
    compute_delta=True
)

irs_mtm_2 = swap_mtm(irs_rate_15y, irs_15y_dates, discounts_steep)

print(f"Swaption: {swaption_notional * swaption_price_2}")
print(f"IRS: {notional_15y * irs_mtm_2}")

ptf_2 = swaption_notional * swaption_price_2 + notional_15y * irs_mtm_2

print(f'P&L: {ptf_2-ptf_0:,.2f}')

Swaption: 56883339.5877602
IRS: 97536.2081717866
P&L: -81,841.63


Between the two, the latter is way less sensisitive to a curve steepening. This was somewhat expected because the portfolio with two instruments is almost perfectly hedged, with a DV01 very close to zero. 